# Programming Project #2: Image Quilting

## CS445: Computational Photography - Fall 2020


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import time

# modify to where you store your project data including utils.py
datadir = "/content/drive/My Drive/CS 180/cs180_final_proj/"

utilfn = datadir + "utils.py"
!cp "$utilfn" .
samplesfn = datadir + "samples"
!cp -r "$samplesfn" .
import utils

In [ ]:
from utils import cut # default cut function for seam finding section

### Part I: Randomly Sampled Texture (10 pts)

In [ ]:
# sample patches randomly
def img_random_sample(sample, patch_size):
    image_size = sample.shape
    x1 = random.randint(0, image_size[0]-patch_size-1)
    y1 = random.randint(0, image_size[1]-patch_size-1)
    x2, y2 = x1+patch_size, y1+patch_size

    patch = sample[x1:x2, y1:y2, :]
    return patch

In [ ]:
def quilt_random(sample, out_size, patch_size):
    """
    Randomly samples square patches of size patchsize from sample in order to create an output image of size outsize.

    :param sample: numpy.ndarray   The image you read from sample directory
    :param out_size: int            The width of the square output image
    :param patch_size: int          The width of the square sample patch
    :return: numpy.ndarray
    """

    num_patch_samples = out_size // patch_size
    out_img = np.zeros((out_size, out_size, 3))

    for i in range(num_patch_samples):
      for j in range(num_patch_samples):
        patch = img_random_sample(sample, patch_size)
        out_img[i*patch_size : (i+1) * patch_size, j*patch_size : (j+1) * patch_size, :] = patch/255.0

    return out_img


In [ ]:
sample_img_fn = 'samples/bricks_small.jpg' # feel free to change
sample_img = cv2.cvtColor(cv2.imread(sample_img_fn), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.savefig('method_1_original.jpeg', bbox_inches='tight')
plt.show()

for i in range(1,3):
  out_size = 200  # change these parameters as needed
  patch_size = 15
  res = quilt_random(sample_img, out_size, patch_size)

  if res is not None:
      plt.imshow(res)
      plt.title('Method 1 (Randomly Sampled Texture), Run '+str(i))
      plt.savefig('method_1_sample_'+str(i)+'.jpeg', bbox_inches='tight')
      plt.show()


### Part II: Overlapping Patches (30 pts)

Create a function quilt_simple(sample, out_size, patch_size, overlap, tol) that randomly samples square patches of size patch_size from a sample in order to create an output image of size out_size. Start by sampling a random patch for the upper-left corner. Then sample new patches to overlap with existing ones. For example, the second patch along the top row will overlap by patch_size pixels in the vertical direction and overlap pixels in the horizontal direction. Patches in the first column will overlap by patch_size pixels in the horizontal direction and overlap pixels in the vertical direction. Other patches will have two overlapping regions (on the top and left) which should both be taken into account. Once the cost of each patch has been computed, randomly choose on patch whose cost is less than a threshold determined by tol (see description of choose_sample below).

After a patch is sampled, its pixels should be copied directly into the corresponding position in the output image. Note that it is very easy to make alignment mistakes when computing the cost of each patch, sampling a low-cost patch, and copying the patch from the source to the output. Use an odd value for patch_size so that its center is well-defined. Be sure to thoroughly debug, for example, by checking that the overlapping portion of the copied pixels has the same SSD as the originally computed cost. As a sanity check, try generating a small texture image with tol=1, with the first patch sampled from the upper-left of the source image. This should produce a partial copy of the source image. Once you have this function working, save a result (with higher tolerance for more stochastic texture) generated from the same sample as used for the random method.

`ssd_patch` performs template matching with the overlapping region, computing the cost of sampling each patch, based on the sum of squared differences (SSD) of the overlapping regions of the existing and sampled patch. I suggest using a masked template. The template is the patch in the current output image that is to be filled in (many pixel values will be 0 because they are not filled in yet). The mask has the same size as the patch template and has values of 1 in the overlapping region and values of 0 elsewhere. The SSD of the masked template with the input texture image can be computed efficiently using filtering operations. Suppose I have a template T, a mask M, and an image I: then, ssd_cost = ((M*T)**2).sum() - 2 * cv2.filter2D(I, ddepth=-1, kernel = M*T) + cv2.filter2D(I ** 2, ddepth=-1, kernel=M). You can compute SSD in this way for each channel and sum the costs over channels. Each pixel of the ssd_cost gives you the cost for sampling a patch centered around that pixel.

`choose_sample` should take as input a cost image (each pixel's value is the cost of selecting the patch centered at that pixel) and select a randomly sampled patch with low cost. It's recommended to sort the costs and choose of of the tol smallest costs. So if tol=1, the lowest cost will always be chosen (this is a good way to debug but mainly copies the input texture). If tol=3, one of the three lowest cost patches will be chosen.

In [ ]:
def ssd_patch(output, patch_coords, patch):
    """
    Find SSD between selected patch from sample and
    output: output image
    patch_coords: (i,j) upper left coordinates of patch
    patch: selected patch from texture image (sample)
    """
    uY, uX = patch_coords
    output_patch = output[uY : uY + patch.shape[0], uX : uX + patch.shape[1], :]
    mask = output_patch.copy()
    mask[mask > 0] = 1
    maskedPatch = patch.copy() * mask
    diff = output_patch - maskedPatch
    return np.sum(diff ** 2)

In [ ]:
def choose_sample(cost_image, tol):
    # flatten cost image and sort
    flat_costs = cost_image.ravel()
    sorted_indices = np.argsort(flat_costs)

    random_tol = np.random.randint(tol)
    # print("random tol", random_tol)
    chosen_index = sorted_indices[random_tol]
    chosen_row = chosen_index // cost_image.shape[1]
    chosen_col = chosen_index % cost_image.shape[1]

    # print(chosen_row, chosen_col)

    return chosen_row, chosen_col

In [ ]:
def quilt_simple(sample, out_size, patch_size, overlap, tol):
    """
    Create a quilted image from a sample by randomly sampling patches.

    :param sample: Source image from which patches are sampled.
    :param out_size: Size of the output image.
    :param patch_size: Size of each patch.
    :param overlap: Overlap size between patches.
    :param tol: Tolerance for choosing patches based on SSD cost.
    :return: Quilted image.
    """
    output = np.zeros((out_size, out_size, sample.shape[2]), dtype=sample.dtype)

    for i in range(0, out_size - patch_size, patch_size - overlap):
        for j in range(0, out_size - patch_size, patch_size - overlap):
            matrix = np.zeros((sample.shape[0] - patch_size, sample.shape[1] - patch_size))
            for y in range(matrix.shape[0]):
              for x in range(matrix.shape[1]):
                patch = sample[y:y+patch_size, x:x+patch_size]
                matrix[y,x] = ssd_patch(output, (i,j), patch)
            uY, uX = choose_sample(matrix, tol)
            patch_chosen = sample[uY:uY+patch_size, uX:uX+patch_size]
            output[i:i+patch_size,j:j+patch_size] = patch_chosen
    return output



In [ ]:
sample_img_fn = 'samples/bricks_small.jpg'
sample_img = cv2.cvtColor(cv2.imread(sample_img_fn), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.show()

tol_lst = [5,10,15]

for i in tol_lst:
  out_size = 300  # change these parameters as needed
  patch_size = 50
  overlap = 20
  tol = i

  res = quilt_simple(sample_img, out_size, patch_size, overlap, tol) #feel free to change parameters to get best results
  if res is not None:
      plt.figure(figsize=(10,10))
      plt.title('Method 2 (Overlapping Patches), tol = '+str(i))
      plt.imshow(res)
      plt.savefig('method_2_tol_'+str(i)+'.jpeg', bbox_inches='tight')
      plt.show()

### Part III: Seam Finding (20 pts)


Use the cut function in utils.py (download starter_codes at the top) that finds the min-cost contiguous path from the left to right side of the patch according to the cost indicated by bndcost. The cost of a path through each pixel is the square differences (summed over RGB for color images) of the output image and the newly sampled patch. Note that if a patch has top and left overlaps, you will need to compute two seams and the mask can be defined as the intersection of the masks for each seam (np.logical_and(mask1,mask2)). To find a vertical path, you can apply cut to the transposed patch, e.g., cut(bndcost.T).T.

Create a function quilt_cut that incorporates the seam finding and use it to create a result to compare to the previous two methods.

In [ ]:
def customized_cut(bndcost):
    pass

In [ ]:
def get_patch(sample, output, i, j, tol):
      i_end = min(i + patch_size, out_size)
      j_end = min(j + patch_size, out_size)
      region = output[i:i_end, j:j_end]

      mask = np.zeros_like(region)
      if i > 0:
          mask[:overlap, :] = 1  # overlap on top
      if j > 0:
          mask[:, :overlap] = 1  # overlap on left

      ssd = ssd_patch(region, mask, sample)
      print(ssd.shape)
      patch_i, patch_j = choose_sample(ssd, tol)
      patch_i = min(patch_i, sample.shape[0] - patch_size)
      patch_j = min(patch_j, sample.shape[1] - patch_size)

      patch = sample[patch_i:patch_i + patch_size, patch_j:patch_j + patch_size]
      return patch


In [ ]:
def blend_patches(output, patch, mask, i, j, overlap):
    blended_patch = np.copy(patch)

    if i > 0:
        for row in range(overlap):
            for col in range(patch.shape[1]):
                if mask[row, col]:
                    blended_patch[row, col] = patch[row, col]
                else:
                    blended_patch[row, col] = output[i + row, j + col]

    if j > 0:
        for row in range(patch.shape[0]):
            for col in range(overlap):
                if mask[row, col]:
                    blended_patch[row, col] = patch[row, col]
                else:
                    blended_patch[row, col] = output[i + row, j + col]

    return blended_patch

In [ ]:
def compute_boundary_cost(output, patch, i, j, overlap):
    boundary_cost = np.zeros((patch.shape[0], patch.shape[1]))

    if i > 0:
        top_overlap_output = output[i:i + overlap, j:j + patch.shape[1]]
        top_overlap_patch = patch[:overlap, :]
        boundary_cost[:overlap, :] = np.sum((top_overlap_output - top_overlap_patch)**2, axis=2)

    if j > 0:
        left_overlap_output = output[i:i + patch.shape[0], j:j + overlap]
        left_overlap_patch = patch[:, :overlap]
        boundary_cost[:, :overlap] += np.sum((left_overlap_output - left_overlap_patch)**2, axis=2)

    return boundary_cost


In [ ]:
def quilt_cut(sample, out_size, patch_size, overlap, tol):
    """
    Samples square patches of size patchsize from sample using seam finding in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: float
    :return: numpy.ndarray
    """

    output = np.zeros((out_size, out_size, sample.shape[2]), dtype=sample.dtype)

    for i in range(0, out_size - patch_size, patch_size - overlap):
        for j in range(0, out_size - patch_size, patch_size - overlap):
            matrix = np.zeros((sample.shape[0] - patch_size, sample.shape[1] - patch_size))
            for y in range(matrix.shape[0]):
              for x in range(matrix.shape[1]):
                patch = sample[y:y+patch_size, x:x+patch_size]
                matrix[y,x] = ssd_patch(output, (i,j), patch)
            uY, uX = choose_sample(matrix, tol)
            patch_chosen = sample[uY:uY+patch_size, uX:uX+patch_size]

            if i > 0 or j > 0:
                bndcost = compute_boundary_cost(output, patch_chosen, i, j, overlap)
                if i > 0 and j > 0:
                    mask1 = cut(bndcost)
                    mask2 = cut(bndcost.T).T
                    mask = np.logical_and(mask1, mask2)
                elif i > 0:
                    mask = cut(bndcost)
                else:  # j > 0
                    mask = cut(bndcost.T).T

                patch = blend_patches(output, patch_chosen, mask, i, j, overlap)

            output[i:i+patch_size,j:j+patch_size] = patch_chosen
    return output

In [ ]:
sample_img_fn = 'samples/bricks_small.jpg'
sample_img = cv2.cvtColor(cv2.imread(sample_img_fn), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.show()


out_size = 300  # change these parameters as needed
patch_size = 50
overlap = 20
tol = 3


res = quilt_cut(sample_img, out_size, patch_size, overlap, tol)
if res is not None:
    plt.figure(figsize=(15,15))
    plt.imshow(res)

In [ ]:
sample_img_fn = 'samples/texture.png'
sample_img = cv2.cvtColor(cv2.imread(sample_img_fn), cv2.COLOR_BGR2RGB)
plt.imshow(sample_img)
plt.show()


out_size = 600  # change these parameters as needed
patch_size = 60
overlap = 25
tol = 3


res = quilt_cut(sample_img, out_size, patch_size, overlap, tol)
if res is not None:
    plt.figure(figsize=(15,15))
    plt.imshow(res)

### part IV: Texture Transfer (30 pts)

In [ ]:
# def get_guidance_ssd(guidance_grey, texture_gray, i, j, guidance_patch_size, texture_patch_size):
#     # source: guidance img
#     guidance_ssd = np.zeros(source.shape[:2])
#     target_region = target[i:i + patch_size, j:j + patch_size]
#     for row in range(source.shape[0] - patch_size + 1):
#         for col in range(source.shape[1] - patch_size + 1):
#             source_patch = source[row:row + patch_size, col:col + patch_size]
#             cost = np.sum((source_patch - target_region)**2)
#             guidance_ssd[row, col] = cost
#     return guidance_ssd

In [ ]:
# def ssd_patch(output, patch_coords, patch):
#     """
#     Find SSD between selected patch from sample and
#     output: output image
#     patch_coords: (i,j) upper left coordinates of patch
#     patch: selected patch from texture image (sample)
#     """
#     uY, uX = patch_coords
#     ROI = output[uY : uY + patch.shape[0], uX : uX + patch.shape[1], :]
#     mask = ROI.copy()
#     mask[mask > 0] = 1
#     maskedPatch = patch.copy() * mask
#     diff = ROI - maskedPatch
#     return np.sum(diff ** 2)

In [ ]:
from skimage.transform import resize

def texture_transfer(sample, patch_size, overlap, tol, guidance_im, alpha, texture_gray, guidance_gray,  guidance_gray_blurred, texture_gray_blurred):
    """
    Samples square patches of size patchsize from sample using seam finding in order to create an output image of size outsize.
    Feel free to modify function parameters
    :param sample: numpy.ndarray
    :param patch_size: int
    :param overlap: int
    :param tol: float
    :param guidance_im: target overall appearance for the output
    :param alpha: float 0-1 for strength of target
    :return: numpy.ndarray
    """
    assert guidance_im.shape[0] > sample.shape[0]

    out_size = guidance_im.shape[:2]
    output = np.zeros((out_size[0], out_size[1], sample.shape[2]), dtype=sample.dtype)

    guidance_im = resize(guidance_im, (sample.shape[0] , sample.shape[1]),
                       anti_aliasing=True)


    for i in range(0, out_size[0] - patch_size - overlap, patch_size - overlap):
        for j in range(0, out_size[1] - patch_size - overlap, patch_size - overlap):

            ssd_matrix = np.zeros((guidance_im.shape[0] - patch_size, guidance_im.shape[1] - patch_size))
            guidance_ssd_matrix = np.zeros((guidance_im.shape[0] - patch_size, guidance_im.shape[1] - patch_size))

            for y in range(ssd_matrix.shape[0]):
              for x in range(ssd_matrix.shape[1]):
                target_patch = sample[y:y+patch_size, x:x+patch_size]
                ssd_matrix[y,x] = ssd_patch(output, (i,j), target_patch)

                grey_target_patch = guidance_gray[y:y+patch_size, x:x+patch_size]
                grey_texture_patch = texture_gray[y:y+patch_size, x:x+patch_size]
                grey_ssd = np.sum((grey_target_patch - grey_texture_patch) **2)

                blurred_grey_target_patch = guidance_gray_blurred[y:y+patch_size, x:x+patch_size]
                blurred_grey_texture_patch = texture_gray_blurred[y:y+patch_size, x:x+patch_size]
                blurred_grey_ssd = np.sum((blurred_grey_target_patch - blurred_grey_texture_patch) **2)

                guidance_ssd_matrix[y,x] = (0.5 * grey_ssd) + (0.5 * blurred_grey_ssd)

            # # normalize
            ssd_matrix = ssd_matrix / np.linalg.norm(ssd_matrix)
            guidance_ssd_matrix = guidance_ssd_matrix/ np.linalg.norm(guidance_ssd_matrix)

            combined_ssd_matrix = (alpha * ssd_matrix) + ((1-alpha) * guidance_ssd_matrix)
            uY, uX = choose_sample(combined_ssd_matrix, tol)

            patch_chosen = sample[uY:uY+patch_size, uX:uX+patch_size]

            if i > 0 or j > 0:
                bndcost = compute_boundary_cost(output, patch_chosen, i, j, overlap)
                if i > 0 and j > 0:
                    mask1 = cut(bndcost)
                    mask2 = cut(bndcost.T).T
                    mask = np.logical_and(mask1, mask2)
                elif i > 0:
                    mask = cut(bndcost)
                else:  # j > 0
                    mask = cut(bndcost.T).T

                patch_chosen = blend_patches(output, patch_chosen, mask, i, j, overlap)

            output[i:i+patch_size,j:j+patch_size] = patch_chosen
        plt.imshow(output)
        plt.show()

    return output


In [ ]:
# load/process appropriate input texture and guidance images
texture_img_path = 'samples/sketch.tiff'
guidance_img_path = 'samples/feynman.tiff'

texture_img = cv2.cvtColor(cv2.imread(texture_img_path), cv2.COLOR_BGR2RGB)
texture_img = cv2.resize(texture_img, (0,0), fx=0.75, fy=0.75)

texture_gray = cv2.imread(texture_img_path,0)
print(texture_gray.shape)
texture_gray = cv2.resize(texture_gray, (0,0), fx=0.75, fy=0.75)
texture_gray = cv2.normalize(texture_gray, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
texture_gray_blurred = cv2.GaussianBlur(texture_gray,(25,25),0)

plt.imshow(texture_img)
plt.show()

guidance_img = cv2.cvtColor(cv2.imread(guidance_img_path), cv2.COLOR_BGR2RGB)
guidance_gray = cv2.imread(guidance_img_path, 0)
guidance_gray = cv2.normalize(guidance_gray, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
guidance_gray_blurred = cv2.GaussianBlur(guidance_gray,(25,25),0)



plt.imshow(guidance_img)
plt.show()


patch_size = 15
overlap = 4
tol = 1
alpha = 0.1 #texture weight
res = texture_transfer(texture_img, patch_size, overlap, tol, guidance_img, alpha, texture_gray, guidance_gray, guidance_gray_blurred, texture_gray_blurred)

plt.figure(figsize=(15,15))
plt.imshow(res)
plt.show()

### Bells & Whistles

(10 pts) Create and use your own version of cut.m. To get these points, you should create your own implementation without basing it directly on the provided function (you're on the honor code for this one).

You can simply copy your customized_cut(bndcost) into the box below so that it is easier for us to grade

(15 pts) Implement the iterative texture transfer method described in the paper. Compare to the non-iterative method for two examples.

(up to 20 pts) Use a combination of texture transfer and blending to create a face-in-toast image like the one on top. To get full points, you must use some type of blending, such as feathering or Laplacian pyramid blending.

(up to 40 pts) Extend your method to fill holes of arbitrary shape for image completion. In this case, patches are drawn from other parts of the target image. For the full 40 pts, you should implement a smart priority function (e.g., similar to Criminisi et al.).